 # 天池贷款违约预测竞赛 - Stacking Ensemble (进阶版)



 **竞赛链接**：[天池-零基础入门金融风控-贷款违约预测](https://tianchi.aliyun.com/competition/entrance/531830/information)



 **核心策略**：Stacking 集成



目标是利用不同模型的优势，通过“取长补短”来提升预测精度。



 * **Level 0 (专家团)**：负责从原始数据中提取规律。

     * **LightGBM**: 梯度提升树，训练速度快，精度高。

     * **XGBoost**: 老牌强者，稳健性极佳。

     * **CatBoost**: 擅长处理类别特征，与其他两个树模型差异性较大。

 * **Level 1 (裁判员)**：负责综合专家的意见。

     * **Logistic Regression (LR)**: 简单的线性模型，用来给三个专家的预测结果加权。



 **主要流程**：

 1.  **数据准备**：读取并预处理数据。

 2.  **第一层训练 (Level 0)**：三位专家分别进行 5折交叉验证，生成“元特征”(Meta-features)。

 3.  **相关性分析**：观察三位专家的意见是否一致（意见越不一致，融合效果往往越好）。

 4.  **第二层训练 (Level 1)**：LR 根据元特征训练，得到最终的加权策略。

 5.  **结果输出**：生成提交文件。

In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import time
import warnings
import re

# 设置显示选项：显示所有列，方便查看数据
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
warnings.filterwarnings("ignore")


 ## 1. 数据读取与预处理



 这里复用了之前验证过的稳健预处理逻辑。我们先合并训练集和测试集，统一处理完后再拆分。

In [2]:
def load_data(train_path, test_path):
    print(f"正在读取数据...\n训练集: {train_path}\n测试集: {test_path}")
    train_df = pd.read_csv(train_path)
    test_df = pd.read_csv(test_path)
    print(f"原始训练集形状: {train_df.shape}")
    print(f"原始测试集形状: {test_df.shape}")
    return train_df, test_df

# 请确保当前目录下有这两个文件
TRAIN_FILE = "./train.csv"
TEST_FILE = "./testA.csv"

train_data, test_data = load_data(TRAIN_FILE, TEST_FILE)

# 提取并分离标签
target = train_data['isDefault']
train_data = train_data.drop(['isDefault'], axis=1)

# 合并数据
data = pd.concat([train_data, test_data], ignore_index=True)

# --- 缺失值填充 ---
# 1. 离散/文本型 -> 众数填充
# employmentLength 缺失通常代表无工作，填0
data['employmentLength'] = data['employmentLength'].fillna(0)
for col in ['employmentTitle', 'postCode', 'title']:
    if data[col].isnull().sum() > 0:
        data[col] = data[col].fillna(data[col].mode()[0])

# 2. 连续数值型 -> 均值填充
for col in ['dti', 'pubRecBankruptcies', 'revolUtil', 'n11', 'n12']:
    if data[col].isnull().sum() > 0:
        data[col] = data[col].fillna(data[col].mean())

# 3. 剩余匿名特征 -> 众数填充
numerical_features = list(data.select_dtypes(exclude=['object']).columns)
no_name_list = [col for col in numerical_features if col.startswith("n") and data[col].isnull().sum() > 0]
for col in no_name_list:
    data[col] = data[col].fillna(data[col].mode()[0])

# --- 特征工程 ---
# 1. 处理工作年限 (正则提取数字)
def employment_length_to_int(s):
    if pd.isnull(s) or str(s) == 'nan': return 0
    s = str(s).strip()
    if '< 1' in s: return 0
    numbers = re.findall(r'\d+', s)
    return int(numbers[0]) if numbers else 0

data['employmentLength'] = data['employmentLength'].apply(employment_length_to_int)

# 2. 处理最早信用记录 (提取年份)
data['earliesCreditLine'] = data['earliesCreditLine'].apply(lambda s: int(str(s)[-4:]))

# 3. 处理等级 (Label Encoding)
for col in ['grade', 'subGrade']:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col].astype(str))

# 4. 删除日期列 (暂不使用)
if 'issueDate' in data.columns:
    data = data.drop(['issueDate'], axis=1)

# --- 重新拆分数据 ---
train_features = data[:len(target)]
test_features = data[len(target):]

print("\n数据预处理完成！")
print(f"最终训练集特征维度: {train_features.shape}")
print(f"最终测试集特征维度: {test_features.shape}")


正在读取数据...
训练集: ./train.csv
测试集: ./testA.csv
原始训练集形状: (800000, 47)
原始测试集形状: (200000, 46)

数据预处理完成！
最终训练集特征维度: (800000, 45)
最终测试集特征维度: (200000, 45)


 ## 2. 构建 Level 0：专家团 (Base Learners)



 我们定义一个通用的 `get_oof` 函数。这个函数非常关键，它解决了不同模型库（Library）在最新版本中 API 不兼容的问题。



 * **LightGBM (v4.0+)**: 必须使用 `callbacks=[...]` 来控制早停。

 * **XGBoost (v2.0+)**: 必须在初始化时传入 `early_stopping_rounds`，不能在 fit 中传。

 * **CatBoost**: 依然支持在 fit 中传入 `early_stopping_rounds`。

In [3]:
nfold = 5
skf = StratifiedKFold(n_splits=nfold, shuffle=True, random_state=2023)

def get_oof(clf, x_train, y_train, x_test, name="Model"):
    """
    通用函数：执行 K折交叉验证，获取 Out-Of-Fold 预测值。
    """
    print(f"\n=== 专家 [{name}] 开始诊断... ===")
    
    # oof_train: 存放训练集每一折作为验证集时的预测结果 (长度 = 训练集行数)
    oof_train = np.zeros((len(x_train),))
    # oof_test_skf: 存放每一折模型对测试集的预测结果 (长度 = 测试集行数 x 折数)
    oof_test_skf = np.zeros((len(x_test), nfold))
    
    start_time = time.time()
    
    for i, (train_index, test_index) in enumerate(skf.split(x_train, y_train)):
        # 切分数据
        kf_x_train = x_train.iloc[train_index]
        kf_y_train = y_train.iloc[train_index]
        kf_x_test = x_train.iloc[test_index]
        kf_y_test = y_train.iloc[test_index]
        
        # 获取模型类型，用于区分不同的 API 调用方式
        model_type = str(type(clf))
        
        # --- 1. LightGBM 处理逻辑 ---
        if 'LGBMClassifier' in model_type:
            callbacks = [lgb.early_stopping(stopping_rounds=50), lgb.log_evaluation(period=0)]
            clf.fit(
                kf_x_train, kf_y_train, 
                eval_set=[(kf_x_test, kf_y_test)], 
                eval_metric='auc', 
                callbacks=callbacks
            )
            
        # --- 2. XGBoost 处理逻辑 ---
        elif 'XGBClassifier' in model_type:
            # XGBoost 2.0+ 早停参数必须在初始化时设置，此处只负责 fit
            clf.fit(
                kf_x_train, kf_y_train, 
                eval_set=[(kf_x_test, kf_y_test)], 
                verbose=False
            )
            
        # --- 3. CatBoost 处理逻辑 ---
        else:
            clf.fit(
                kf_x_train, kf_y_train, 
                eval_set=[(kf_x_test, kf_y_test)], 
                verbose=False, 
                early_stopping_rounds=50
            )
        
        # 预测当前折的验证集
        val_preds = clf.predict_proba(kf_x_test)[:,1]
        oof_train[test_index] = val_preds
        
        # 预测测试集
        oof_test_skf[:,i] = clf.predict_proba(x_test)[:,1]
        
        # 计算并打印当前折的 AUC 分数
        fold_auc = roc_auc_score(kf_y_test, val_preds)
        print(f"Fold {i+1}/{nfold} AUC: {fold_auc:.5f}")

    # 计算整体 CV 分数
    all_auc = roc_auc_score(y_train, oof_train)
    print(f"专家 [{name}] 5折验证总 AUC: {all_auc:.5f}")
    print(f"耗时: {time.time() - start_time:.2f} 秒")
    
    # 测试集结果取平均
    oof_test = oof_test_skf.mean(axis=1)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)


 ### 2.1 训练专家 A：LightGBM

In [4]:
clf_lgb = lgb.LGBMClassifier(
    num_leaves=31, max_depth=-1, learning_rate=0.05, n_estimators=2000,
    objective='binary', metric='auc', verbose=-1, random_state=2023
)
lgb_train, lgb_test = get_oof(clf_lgb, train_features, target, test_features, name="LightGBM")



=== 专家 [LightGBM] 开始诊断... ===
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[898]	valid_0's auc: 0.731468
Fold 1/5 AUC: 0.73147
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1053]	valid_0's auc: 0.72787
Fold 2/5 AUC: 0.72787
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1000]	valid_0's auc: 0.731249
Fold 3/5 AUC: 0.73125
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1066]	valid_0's auc: 0.729213
Fold 4/5 AUC: 0.72921
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1050]	valid_0's auc: 0.729796
Fold 5/5 AUC: 0.72980
专家 [LightGBM] 5折验证总 AUC: 0.72991
耗时: 119.12 秒


 ### 2.2 训练专家 B：XGBoost

 **注意**：针对 XGBoost 2.0+，`early_stopping_rounds` 必须在初始化时传入。

In [5]:
clf_xgb = xgb.XGBClassifier(
    max_depth=6, learning_rate=0.05, n_estimators=2000,
    objective='binary:logistic', eval_metric='auc',
    tree_method='hist',         # 使用直方图加速
    early_stopping_rounds=50,   # 早停策略放在这里
    random_state=2023, n_jobs=-1
)
xgb_train, xgb_test = get_oof(clf_xgb, train_features, target, test_features, name="XGBoost")



=== 专家 [XGBoost] 开始诊断... ===
Fold 1/5 AUC: 0.73323
Fold 2/5 AUC: 0.72879
Fold 3/5 AUC: 0.73269
Fold 4/5 AUC: 0.72983
Fold 5/5 AUC: 0.73132
专家 [XGBoost] 5折验证总 AUC: 0.73116
耗时: 174.35 秒


 ### 2.3 训练专家 C：CatBoost

 CatBoost 对类别特征处理独特，通常能提供差异化的预测视角。

In [6]:
clf_cat = CatBoostClassifier(
    iterations=1000, learning_rate=0.05, depth=6,
    eval_metric='AUC', verbose=False, random_state=2023,
    allow_writing_files=False
)
cat_train, cat_test = get_oof(clf_cat, train_features, target, test_features, name="CatBoost")



=== 专家 [CatBoost] 开始诊断... ===
Fold 1/5 AUC: 0.73082
Fold 2/5 AUC: 0.72714
Fold 3/5 AUC: 0.73160
Fold 4/5 AUC: 0.72900
Fold 5/5 AUC: 0.72993
专家 [CatBoost] 5折验证总 AUC: 0.72969
耗时: 222.12 秒


 ## 3. 构建元特征 (Meta-features) 与相关性分析



 我们将三位专家的预测结果拼接成一张新的表格。这张表只有3列，每一列代表一个专家的看法。



 **为什么看相关性？**

 如果三个模型预测结果一模一样（相关性=1.0），那融合就没有意义了。相关性越低，说明模型关注的数据侧面不同，互补性越强。

In [7]:
# 拼接训练集元特征
x_train_stack = np.concatenate((lgb_train, xgb_train, cat_train), axis=1)
x_train_stack = pd.DataFrame(x_train_stack, columns=['LightGBM', 'XGBoost', 'CatBoost'])

# 拼接测试集元特征
x_test_stack = np.concatenate((lgb_test, xgb_test, cat_test), axis=1)
x_test_stack = pd.DataFrame(x_test_stack, columns=['LightGBM', 'XGBoost', 'CatBoost'])

print("\n=== 元特征 (Meta-features) 前5行预览 ===")
print(x_train_stack.head())

print("\n=== 模型预测结果相关性矩阵 (Correlation Matrix) ===")
# 观察模型之间的相似度，越低越好（通常 0.9 以上都很高了，但只要不是 1.0 就有提升空间）
print(x_train_stack.corr())



=== 元特征 (Meta-features) 前5行预览 ===
   LightGBM   XGBoost  CatBoost
0  0.313510  0.268663  0.308155
1  0.281933  0.265492  0.251009
2  0.371695  0.401617  0.399574
3  0.052989  0.051469  0.063408
4  0.352887  0.338223  0.356561

=== 模型预测结果相关性矩阵 (Correlation Matrix) ===
          LightGBM   XGBoost  CatBoost
LightGBM  1.000000  0.982753  0.984224
XGBoost   0.982753  1.000000  0.976763
CatBoost  0.984224  0.976763  1.000000


 ## 4. Level 1：裁判员 (LR Meta-learner)



 使用逻辑回归（LR）作为裁判。

 LR 会根据元特征学习一个公式：

 $$Final\_Score = w_1 \cdot LGB + w_2 \cdot XGB + w_3 \cdot Cat + Bias$$



 * **w (权重)**：如果某个专家特别准，LR 会给他更高的权重。

 * **Bias (截距)**：用于修正整体概率的偏移。

In [8]:
print("\n=== 裁判员 (LR) 开始综合意见... ===")

# 初始化 LR (不需要标准化，因为输入的概率值本身就在 0-1 之间且尺度一致)
meta_learner = LogisticRegression(random_state=2023)
meta_learner.fit(x_train_stack, target)

# 获取权重
weights = meta_learner.coef_[0]
bias = meta_learner.intercept_[0]

print("\n=== 最终权重分配 (裁判的决定) ===")
print(f"Bias (基础偏置): {bias:.4f}")
print(f"LightGBM 权重: {weights[0]:.4f}")
print(f"XGBoost  权重: {weights[1]:.4f}")
print(f"CatBoost 权重: {weights[2]:.4f}")

# 生成最终预测
final_preds = meta_learner.predict_proba(x_test_stack)[:, 1]

# 计算 Stacking 在训练集上的表现
train_preds_final = meta_learner.predict_proba(x_train_stack)[:, 1]
score = roc_auc_score(target, train_preds_final)

print(f"\n>>> Stacking 融合后最终 CV AUC: {score:.5f} <<<")



=== 裁判员 (LR) 开始综合意见... ===

=== 最终权重分配 (裁判的决定) ===
Bias (基础偏置): -2.7471
LightGBM 权重: 1.9781
XGBoost  权重: 2.0545
CatBoost 权重: 1.9357

>>> Stacking 融合后最终 CV AUC: 0.73152 <<<


 ## 5. 生成提交文件

In [9]:
submission = pd.DataFrame()
# 检查是否有 ID 列，如果没有则使用索引
submission['id'] = test_data['id'] if 'id' in test_data.columns else range(len(final_preds))
submission['isDefault'] = final_preds

output_file = 'submit_stack.csv'
submission.to_csv(output_file, index=False)

print(f"任务完成！提交文件已保存至: {output_file}")

任务完成！提交文件已保存至: submit_stack.csv
